In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from torchvision.io import read_image
ASSETS_DIRECTORY = "assets"

imgs_path = os.path.join(ASSETS_DIRECTORY, "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Images/")
imgs = os.listdir(imgs_path)

masks_path = os.path.join(ASSETS_DIRECTORY, "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Masks/")
Masks = os.listdir(masks_path)
boxes_path =  os.path.join(ASSETS_DIRECTORY, "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Boxes/")
box_dir = os.listdir(boxes_path)


In [4]:
def findIndex(i):
  name = imgs[i].split(".")[0]
  name = name + "_mask.png"
  index = Masks.index(name)
  return index

In [6]:
from torchvision.ops import masks_to_boxes
import torch

In [7]:
def pathSaveBoxe(imgName):
  return boxes_path + imgName[:-4] + "_box.pt"

In [8]:
def deleteExistingMask():
  box_dir2 = []
  for i in range(len(box_dir)):
    box_dir2.append(box_dir[i].replace("_box.pt", "")) 

  files2 = []
  for i in range(len(imgs)):
    files2.append(imgs[i].replace(".jpg", ""))

  set1 = set(box_dir2)
  set2 = set(files2)

  elementi_comuni = set1.intersection(set2)

  elementi_comuni_lista = list(elementi_comuni)

  for elemento in elementi_comuni_lista:
      elemento = elemento + ".jpg"
      imgs.remove(elemento)

  return len(imgs)

In [ ]:
if len(Masks) != len(imgs):
  print("Differenza tra il numero di immagini e di maschere errore")
elif deleteExistingMask() == 0:
  print("Tutte le box sono già disponibili")
else:
  for i in range(len(imgs)):
    index = findIndex(i) 
    img_path = imgs_path + imgs[i]
    mask_path = masks_path + Masks[index]
    
    img = read_image(img_path)
    mask = read_image(mask_path)



    obj_ids = torch.unique(mask)


    obj_ids = obj_ids[1:]

    masks = mask == obj_ids[:, None, None]

    boxes = masks_to_boxes(masks)
    
    torch.save(boxes, pathSaveBoxe(imgs[i]))


# Per visualizzare le Immagini

In [ ]:
#da eliminare dopo le prove
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes #da eliminare dopo le prove

import torchvision.transforms.functional as F


ASSETS_DIRECTORY = "assets"

plt.rcParams["savefig.bbox"] = "tight"


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols= len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [ ]:
if (len(Masks) != len(imgs)) or len(box_dir) != len(imgs):
  print("Le directory non hanno lo stesso numero di elementi")
else:
  images = []
  Boxes = []
  for i in range(len(imgs)):
    print("Numero di elementi caricati: ", i+1)
    
    index_b = findIndex(i) #funzione creata precedentemente 
    img_path = imgs_path + imgs[i]
    #mask_path = masks_path + Masks[index_m]
    box_path = boxes_path + box_dir[index_b]
    print(img_path)
    #print(mask_path)
    print(box_path)
    img = read_image(img_path)
    images.append(img)
    #mask = read_image(mask_path)
    box = torch.load(box_path)
    Boxes.append(box)